In [1]:
import torch

## Creating Layers

In [2]:
class DenseLayer:
  # Layer initialization
  def __init__(self, n_inputs, n_neurons):
    # Initialize weights and biases
    self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
    self.biases = torch.zeros((1, n_neurons))

  # Forward pass
  def forward(self, inputs):
    # Calculate output values from inputs, weights and biases
    self.output = torch.matmul(inputs, self.weights) + self.biases

## Activation Functions

In [10]:
class Activation_ReLU:
  # Forward pass
  def forward(self, inputs):
    self.output = torch.max(torch.tensor(0),inputs)
class Activation_Sigmoid:
  # Forward pass
  def forward(self, inputs):
    self.output = 1 / (1 + torch.exp(inputs*-1))
class Activation_Softmax:
  # Forward pass
  def forward(self, inputs):
    # Get unnormalized probabilities
    exp_values = torch.exp(inputs - torch.max(inputs, axis=1, keepdim=True).values)
    # Normalize them for each sample
    probabilities = exp_values / torch.sum(exp_values, axis=1, keepdim=True)
    self.output = probabilities

## Loss and accuracy

In [11]:
class Loss_CategoricalCrossentropy() :
  # Forward pass
  def forward(self, y_pred, y_true):
    samples = len(y_pred)
    # Clip data to prevent division by 0
    # Clip both sides to not drag mean towards any value
    y_pred_clipped = torch.clip(y_pred, 1e-8, 1 - 1e-8)
    # only if categorical labels
    if len(y_true.shape) == 1:
      correct_confidences = y_pred_clipped[range(samples), y_true]
    # Mask values - only for one-hot encoded labels
    elif len(y_true.shape) == 2:
      correct_confidences = torch.sum(y_pred_clipped * y_true, axis=1)
    log_loss = -torch.log(correct_confidences)
    data_loss = torch.mean(log_loss)
    return data_loss
class Accuracy():
  def calculate(self, y_pred, y_true):
    predictions = torch.argmax(y_pred, axis=1)
    if len(y_true.shape) == 2:
      y_true = torch.argmax(y_true, axis=1)
    accuracy = torch.mean((predictions == y_true).float())
    return accuracy

In [8]:
X = torch.tensor([0.1, 0.5])
y = torch.tensor([0.05,0.2, 0.45, 0.3])

In [9]:
hidden_layer_1 = DenseLayer(2, 4)
activation1 = Activation_ReLU()
output_layer = DenseLayer(4, 4)
activation2 = Activation_Sigmoid()

In [20]:
def forward_pass(X):
  hidden_layer_1.forward(X)
  activation1.forward(hidden_layer_1.output)
  output_layer.forward(activation1.output)
  activation2.forward(output_layer.output)
  return activation2.output

In [16]:
def back_prop(fp):
    lr = torch.tensor(0.01)

    # Calculate gradients for the output layer
    back1 = (fp[0][0] - y[0]) * (1 - fp[0][0]) * fp[0][0]
    back2 = (fp[0][1] - y[1]) * (1 - fp[0][1]) * fp[0][1]
    back3 = (fp[0][2] - y[2]) * (1 - fp[0][2]) * fp[0][2]
    back4 = (fp[0][3] - y[3]) * (1 - fp[0][3]) * fp[0][3]

    # Update weights and biases for the output layer
    output_layer.weights[0][0] -= lr * back1 * activation1.output[0][0]
    output_layer.weights[0][1] -= lr * back2 * activation1.output[0][1]
    output_layer.weights[0][2] -= lr * back3 * activation1.output[0][2]
    output_layer.weights[0][3] -= lr * back4 * activation1.output[0][3]
    output_layer.biases[0][0] -= lr * back1
    output_layer.biases[0][1] -= lr * back2
    output_layer.biases[0][2] -= lr * back3
    output_layer.biases[0][3] -= lr * back4

    # Calculate gradients for the hidden layer
    hidden_back1 = back1 * output_layer.weights[0][0] * (1 if hidden_layer_1.output[0][0] > 0 else 0)
    hidden_back2 = back2 * output_layer.weights[0][1] * (1 if hidden_layer_1.output[0][0] > 0 else 0)
    hidden_back3 = back3 * output_layer.weights[0][2] * (1 if hidden_layer_1.output[0][1] > 0 else 0)
    hidden_back4 = back4 * output_layer.weights[0][3] * (1 if hidden_layer_1.output[0][1] > 0 else 0)

    # Update weights and biases for the hidden layer
    hidden_layer_1.weights[0][0] -= lr * (hidden_back1 * X[0] + hidden_back2 * X[0])
    hidden_layer_1.weights[0][1] -= lr * (hidden_back1 * X[1] + hidden_back2 * X[1])
    hidden_layer_1.weights[1][0] -= lr * (hidden_back3 * X[0] + hidden_back4 * X[0])
    hidden_layer_1.weights[1][1] -= lr * (hidden_back3 * X[1] + hidden_back4 * X[1])
    hidden_layer_1.biases[0][0] -= lr * (hidden_back1 + hidden_back3)
    hidden_layer_1.biases[0][1] -= lr * (hidden_back2 + hidden_back4)


In [17]:
def error_calculation(y_true, y_pred):
  return torch.mean(0.5*(y_true - y_pred)**2)

In [44]:
loss = 0.0001
y_pred = forward_pass(X)
err = error_calculation(y, y_pred)
print("Initial loss:", err)
print("Initial prediction:",y_pred)
rotation = 0
while err > loss and rotation < 100000:
  back_prop(y_pred)
  y_pred = forward_pass(X)
  err = error_calculation(y, y_pred)
  rotation += 1
print("Final loss:", err)
print("Final prediction:",y_pred)
print("Target value:",y)

Initial loss: tensor(9.9994e-05)
Initial prediction: tensor([[0.0783, 0.2006, 0.4500, 0.3000]])
Final loss: tensor(9.9994e-05)
Final prediction: tensor([[0.0783, 0.2006, 0.4500, 0.3000]])
Target value: tensor([0.0500, 0.2000, 0.4500, 0.3000])


In [45]:
acc = Accuracy()
accuracy = acc.calculate(y_pred, y.unsqueeze(0))
print("The accuracy is:", accuracy)

The accuracy is: tensor(1.)
